## Showing that finding a path in a Directed Graph can be gotten in Polynomial Time

In [3]:
from typing import List

def get_nodes(edges: List[List[str]]):
    nodes = []

    for edge in edges:

        if edge[0] not in nodes:
            nodes.append(edge[0])

        if edge[1] not in nodes:
            nodes.append(edge[1])
            
    nodes.sort()
    return nodes

## BRUTE-FORCE ALGORITHM

"""

"""

## EFFICIENT ALGORITHM

def checker(edges: List[List[str]], start: str, end: str) -> bool:
    
    marked = {start: True}

    for edge in edges: 
        if marked.get(edge[0]):
            marked[edge[1]] = True
            
    return True if marked.get(end) is not None else False


## Check that a path exists from a to f

edges = [["a", "b"], ["a", "c"], ["b", "e"], ["e", "c"], ["d", "f"]]
print(checker(edges, "a", "e"))

True


## Polynomial reduction function for proof that HAMPATH is NP-complete by showing that 3-SAT can be reduced to it

## Proof of Cook Levin Theorem: SAT is NP-complete

In [2]:
from typing import List, Dict, Optional, Tuple, NewType
from enum import Enum, auto

BLANK = "BLANK"

ACCEPT_STATE = "ACCEPT_STATE"

class DIRECTION(Enum):
    LEFT = auto()
    RIGHT = auto()
    STAY = auto()
    
class STATE(Enum):
    ACCEPT = auto()
    REJECT = auto()
    RUNNING = auto()
    

Transition = NewType("Transition", Tuple[str, Optional[str], DIRECTION])

Transitions = NewType("Transitions", Dict[Tuple[str, str], Transition])

def pretty_print(computations):
    for step, config in enumerate(computations):
        print(f"Computation {step}: {config}")
        
class DTM:
    
    state = STATE.RUNNING
    
    head = []
            
    config: List[str] = []
    
    transitions: Transitions = {}
    
    computations: List[Dict] = []
    
    def __init__(self, start_state: str, start_config: List[str], transitions: Transitions):
        self.head = [start_state, 0]
        start_config.insert(0, start_state)
        self.config = start_config
        self.transitions = transitions
        self.computations = [start_config.copy()]
    
    @staticmethod
    def transition(transitions: Transitions, head: Tuple[str, int], config: List[str], computations: List[List[str]]):
        state = head[0]
        index = head[1]
        config.remove(state)
        symbol = config[index]
                                        
        move = transitions.get((state, symbol))
        
        if move == None:
            state = STATE.REJECT
            return state, head, config, computations
        elif move[0] == ACCEPT_STATE:
            state = STATE.ACCEPT
            return state, head, config, computations
            
        new_state = move[0]
        new_value = move[1]
        direction = move[2]
        if new_value == None:
            if direction == DIRECTION.LEFT:
                new_head_index = index - 1
                if new_head_index == -1:
                    config.insert(0, BLANK)
                    head[0] = new_state
                    head[1] = 0
                else:
                    head[0] = new_state
                    head[1] = new_head_index
            elif direction == DIRECTION.RIGHT:
                new_head_index = index + 1
                if new_head_index > len(config) - 1:
                    config.append(BLANK)
                    head[0] = new_state
                    head[1] = len(config) - 1
                else:
                    head[0] = new_state
                    head[1] = new_head_index

            else:
                pass
        else:
            config[head[1]] = new_value
            if direction == DIRECTION.LEFT:
                new_head_index = index - 1
                if new_head_index == -1:
                    config.insert(0, BLANK)
                    head[0] = new_state
                    head[1] = 0
                else:
                    head[0] = new_state
                    head[1] = new_head_index
            elif direction == DIRECTION.RIGHT:
                new_head_index = index + 1
                if new_head_index > len(config) - 1:
                    config.append(BLANK)
                    head[0] = new_state
                    head[1] = len(config) - 1
                else:
                    head[0] = new_state
                    head[1] = new_head_index
            else:
                pass
        state = STATE.RUNNING
        
        state_index = head[1] - 1
        config.insert(state_index + 1, new_state)
        computations.append(config.copy())
        return state, head, config, computations
                
            
    def run(self):
        (state, head, config, computations) = self.transition(self.transitions, self.head, self.config, self.computations)
        while state == STATE.RUNNING:
            (state, head, config, computations) = self.transition(self.transitions, head, config, computations)
        self.state = state
        self.head = head
        self.config = config
            
start_state = "q0"
start_config = ["0", "0", "0", "0"]
transitions = {
    ("q0", "0"): ("q1", BLANK, DIRECTION.RIGHT),
    ("q1", "x"): ("q1", None, DIRECTION.RIGHT),
    ("q1", "0"): ("q2", "x", DIRECTION.RIGHT),
    ("q2", "x"): ("q2", None, DIRECTION.RIGHT),
    ("q2", "0"): ("q3", None, DIRECTION.RIGHT),
    ("q3", "0"): ("q2", "x", DIRECTION.RIGHT),
    ("q2", BLANK): ("q4", None, DIRECTION.LEFT),
    ("q4", BLANK): ("q1", None, DIRECTION.RIGHT),
    ("q4", "x"): ("q4", None, DIRECTION.LEFT),
    ("q4", "0"): ("q4", None, DIRECTION.LEFT),
    ("q1", BLANK): (ACCEPT_STATE, None, DIRECTION.RIGHT),
}
tm = DTM(start_state, start_config, transitions)
tm.run()
pretty_print(tm.computations)


### Computation TO SAT

class SAT_Prover:
    
    def __init__(self, computations: List[List[str]]):
        self.computations = computations
    
    @staticmethod
    def condition_1(computations: List[List[str]]) -> bool:
        """
        We check that every entry (i, j) in the computation
        has exactly one value. Therefore, we check that each
        entry has one or more values and that each entry cannot
        have 2 values.
        """
        pass
    
    @staticmethod
    def condition_2(computations: List[List[str]]) -> bool:
        pass
    
    @staticmethod
    def condition_3(computations: List[List[str]]) -> bool:
        pass
    
    @staticmethod
    def condition_4(computations: List[List[str]]) -> bool:
        pass

Computation 0: ['q0', '0', '0', '0', '0']
Computation 1: ['BLANK', 'q1', '0', '0', '0']
Computation 2: ['BLANK', 'x', 'q2', '0', '0']
Computation 3: ['BLANK', 'x', '0', 'q3', '0']
Computation 4: ['BLANK', 'x', '0', 'x', 'q2', 'BLANK']
Computation 5: ['BLANK', 'x', '0', 'q4', 'x', 'BLANK']
Computation 6: ['BLANK', 'x', 'q4', '0', 'x', 'BLANK']
Computation 7: ['BLANK', 'q4', 'x', '0', 'x', 'BLANK']
Computation 8: ['q4', 'BLANK', 'x', '0', 'x', 'BLANK']
Computation 9: ['BLANK', 'q1', 'x', '0', 'x', 'BLANK']
Computation 10: ['BLANK', 'x', 'q1', '0', 'x', 'BLANK']
Computation 11: ['BLANK', 'x', 'x', 'q2', 'x', 'BLANK']
Computation 12: ['BLANK', 'x', 'x', 'x', 'q2', 'BLANK']
Computation 13: ['BLANK', 'x', 'x', 'q4', 'x', 'BLANK']
Computation 14: ['BLANK', 'x', 'q4', 'x', 'x', 'BLANK']
Computation 15: ['BLANK', 'q4', 'x', 'x', 'x', 'BLANK']
Computation 16: ['q4', 'BLANK', 'x', 'x', 'x', 'BLANK']
Computation 17: ['BLANK', 'q1', 'x', 'x', 'x', 'BLANK']
Computation 18: ['BLANK', 'x', 'q1', 'x', 

In [9]:
test_list = ["q0", "0", "0", "0", "0"]

res = True

for i, value in enumerate(test_list):
    res = value or value[i + 1]

res

'0'